# EDA

## 1. 개발 환경 구축 및 데이터 로드 (Ground Zero)
> #### **데이터 다운로드 전 필수로 해야하는 일**

👉 **EDA의 첫 단추는 **모든 팀원이 오류 없이 데이터를 읽어올 수 있는 상태**를 만드는 것입니다.**

👉 우리 팀원 5명이 각자의 로컬 `/data/raw/` 폴더에 압축을 푼 `sprint_ai_project1_data` 폴더를 넣어두었다고 가정하겠습니다.

### 1.1 데이터 경로 설정

In [46]:
import pandas as pd
import json
import os
from glob import glob

# [Step 1-1] 현재 위치 기준 데이터 최상위 경로 설정
# '..'은 부모 폴더(pjt-sprint_AI07...)로 올라간다는 뜻입니다.
base_path = os.path.join('..', 'data', 'raw', 'sprint_ai_project1_data')
json_pattern = os.path.join(base_path, 'train_annotations', '*', '*', '*.json')

# [Step 1-2] 모든 JSON 파일 경로 리스트 만들기
# glob은 '유리구슬(glob)'처럼 조건에 맞는 모든 파일을 찾아주는 아주 기특한 도구입니다.
json_files = glob(json_pattern)
print(f"✅ 찾은 JSON 파일 개수: {len(json_files)}개")

✅ 찾은 JSON 파일 개수: 763개


### 문제 : annotations 데이터 폴더의 복잡한 구조
> Kaggle에서 데이터 구조를 살펴보면, `train_images`, `test_images` 폴더는 매우 단순하다.  
> 반면, `train_annotations` 데이터는 상당히 깊고 복잡한 구조를 가졌다.

## 1.2 데이터 구조의 규칙성 발견 (Discovery)
- 이가 없으면 잇몸으로, 하나하나 까본다.
- **참고** 사진 1장에 알약이 4개가 배치된 데이터도 있고, 3개가 배치된 데이터도 존재했다.

    ```powershell
    🗂️train_annotaionts  
    │
    ├── 🗂️K-001900-016548-019607-029451_json   --> 사진 1장에 알약이 4개라면 6자리의 숫자코드가 4개
    │   │
    │   │   --> K-[1번째 약넘버]-[2번째 약넘버]-[3번째 약넘버]-[4번째 약넘버]_json  -> 제목이 나타내는 정보   
    │   │
    │   └── 🗂️K-001900 --> 1번째 약넘버(.json 내부에서 "drug_N: 알약코드" 혹은 "dl_mapping_code: 제품코드"에 표기됨)
    │       │
    │       ├──📄K-001900-016548-019607-029451_0_2_0_2_70_000_200.json  
    │       ├──📄K-001900-016548-019607-029451_0_2_0_2_75_000_200.json  
    │       └──📄K-001900-016548-019607-029451_0_2_0_2_90_000_200.json  
    │
    │           --> 원─핫 인코딩 된 클래스 같음 (추정)
    │    
    │           --> 0_2_0_2 (불확실) : 0 = "drug_S: 알약상태（정상）"
    │                                 2 = "back_color: 촬영배경（연회색 배경）",
    │                                 0 = "drug_dir: 알약방향（앞면）"
    │                                 2 = "light_color: 촬영조명（주백색）"
    │    
    │           --> (70,75,95)_000_200 (확실) : 70,75,95 = "camera_la: 카메라 위도"
    │                                               000 = "camera_la: 카메라 경도"
    │                                               200 = "size: 이미지 사이즈"
    │............  
    ```


### 1.1.1 데이터 샘플 구조 확인 (Sample Check)

In [47]:
if json_files:
    with open(json_files[0], 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
        print("\n🔍 JSON 데이터 샘플 1번째 구조:")
        print(json.dumps(sample_data, indent=4, ensure_ascii=False)[:]) # [:50] -> 앞부분만 살짝 보기
else:
    print("❌ 파일을 찾지 못했습니다. 경로를 다시 확인해주세요!")


🔍 JSON 데이터 샘플 1번째 구조:
{
    "images": [
        {
            "file_name": "K-001900-016548-019607-029451_0_2_0_2_70_000_200.png",
            "width": 976,
            "height": 1280,
            "imgfile": "K-001900-016548-019607-029451_0_2_0_2_70_000_200.png",
            "drug_N": "K-001900",
            "drug_S": "정상알약",
            "back_color": "연회색 배경",
            "drug_dir": "앞면",
            "light_color": "주백색",
            "camera_la": 70,
            "camera_lo": 0,
            "size": 200,
            "dl_idx": "1900",
            "dl_mapping_code": "K-001900",
            "dl_name": "보령부스파정 5mg",
            "dl_name_en": "Buspar Tab. 5mg Boryung",
            "img_key": "http://connectdi.com/design/img/drug/1Mxwka5v0lL.jpg",
            "dl_material": "부스피론염산염",
            "dl_material_en": "Buspirone Hydrochloride",
            "dl_custom_shape": "정제, 저작정",
            "dl_company": "보령제약(주)",
            "dl_company_en": "Boryung",
            "di_company_mf": "",


In [49]:
if json_files:
    with open(json_files[762], 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
        print("\n🔍 JSON 데이터 샘플 2번째 구조:")
        print(json.dumps(sample_data, indent=4, ensure_ascii=False)[:]) # [:50] -> 앞부분만 살짝 보기
else:
    print("❌ 파일을 찾지 못했습니다. 경로를 다시 확인해주세요!")


🔍 JSON 데이터 샘플 2번째 구조:
{
    "images": [
        {
            "file_name": "K-003544-004543-016548-027993_0_2_0_2_90_000_200.png",
            "width": 976,
            "height": 1280,
            "imgfile": "K-003544-004543-016548-027993_0_2_0_2_90_000_200.png",
            "drug_N": "K-027993",
            "drug_S": "정상알약",
            "back_color": "연회색 배경",
            "drug_dir": "앞면",
            "light_color": "주백색",
            "camera_la": 90,
            "camera_lo": 0,
            "size": 200,
            "dl_idx": "27993",
            "dl_mapping_code": "K-027993",
            "dl_name": "졸로푸트정 100mg",
            "dl_name_en": "Zoloft Tab. 100mg",
            "img_key": "http://connectdi.com/design/img/drug/154599025952100024.jpg",
            "dl_material": "설트랄린염산염",
            "dl_material_en": "Sertraline Hydrochloride",
            "dl_custom_shape": "정제, 저작정",
            "dl_company": "한국화이자제약(주)",
            "dl_company_en": "Pfizer Korea",
            "di_compa

### 규칙성 발견 (Discovery)
* JSON 파일을 일일이 열어보던 중, **파일명과 폴더 구조에 일정한 규칙(Schema)** 이 숨어있음을 발견했습니다.

* 굳이 무거운 JSON 파일을 열지 않아도, **파일 경로(Path)만 분석하면** 핵심 정보를 얻을 수 있다는 사실을 확인했습니다.

### 🕵️‍♂️ 발견된 규칙 (Naming Convention)
* **계층 구조:** `train_annotations` > `약제 조합 폴더` > `개별 약제 코드 폴더` > `JSON 파일`

* **핵심 인사이트:**

    1. **정답 라벨(Class):** JSON 파일이 들어있는 **상위 폴더명** 이 곧 약의 코드(`drug_N`)입니다.

    2. **메타 데이터:** 파일명의 뒷부분(`_0_2_0_2_...`)에 촬영 환경 정보가 순서대로 적혀 있습니다.

## 1.3 데이터 파싱 및 통합 데이터프레임 생성 (Implementation)
- 발견한 규칙을 바탕으로 **전체 데이터(763개)를 빠르게 로드하는 파싱 함수**를 구현합니다.

> ### 💡DE 전략 (Strategy)
* **Lazy Loading:** 모든 JSON을 `open()` 하지 않고, `glob`으로 가져온 **파일 경로 문자열(String)** 만 파싱하여 속도를 높입니다.  

* **Robust Parsing:** 파일명 앞부분의 길이가 달라질 수 있으므로, 고정된 길이를 가진 **뒤쪽(Suffix)부터 역순으로 인덱싱** 하여 에러를 방지합니다.

In [55]:
import os
import pandas as pd
from glob import glob

# [Step 1] 데이터 경로 리스트업
base_dir = os.path.join('..', 'data', 'raw', 'sprint_ai_project1_data', 'train_annotations')
json_files = glob(os.path.join(base_dir, '**', '*.json'), recursive=True)

# [Step 2] '찾아낸 규칙'을 적용한 파싱 함수
def parse_metadata_from_path(file_path):
    # 경로를 '/' 기준으로 쪼갭니다. (Windows는 '\\' 자동 처리)
    parts = os.path.normpath(file_path).split(os.sep)
    
    # 1. 정답 라벨 (폴더명에서 추출) -> "K-001900"
    # 파일명 바로 위의 폴더가 주인공이라는 규칙 적용!
    label_code = parts[-2] 
    
    # 2. 파일명에서 메타데이터 추출
    filename = parts[-1]
    # 확장자(.json) 제거 후 '_' 기준으로 쪼개기
    fname_no_ext = os.path.splitext(filename)[0]
    tokens = fname_no_ext.split('_')
    
    # K-코드뭉치 부분이 어디까지인지 모르므로, 뒤에서부터 인덱싱하는 게 안전합니다.
    # 예: ..._0_2_0_2_70_000_200.json
    try:
        meta = {
            'file_path': file_path,
            'label_code': label_code,         # 폴더명이 곧 정답!
            'size': int(tokens[-1]),          # 200
            'camera_lo': int(tokens[-2]),     # 000
            'camera_la': int(tokens[-3]),     # 70
            'light_color': int(tokens[-4]),   # 2
            'drug_dir': int(tokens[-5]),      # 0
            'back_color': int(tokens[-6]),    # 2
            'drug_S': int(tokens[-7])         # 0
        }
        return meta
    except IndexError:
        # 혹시라도 규칙에 안 맞는 파일이 있으면 에러 대신 None 반환
        return None

# [Step 3] 고속 데이터프레임 생성
parsed_data = []
for path in json_files:
    info = parse_metadata_from_path(path)
    if info:
        parsed_data.append(info)

df_meta = pd.DataFrame(parsed_data)

print(f"✅ 총 {len(df_meta)}개의 데이터 메타정보 추출 완료!")
print("JSON을 열지 않고 파일명 규칙만으로 만든 데이터프레임입니다:")
display(df_meta.head())

# [검증] 잘 뽑혔는지 확인 (각 컬럼별 고유값 확인)
print("\n📊 데이터 분포 요약:")
print(df_meta[['drug_dir', 'light_color', 'camera_la']].apply(pd.Series.value_counts))

✅ 총 763개의 데이터 메타정보 추출 완료!
JSON을 열지 않고 파일명 규칙만으로 만든 데이터프레임입니다:


,file_path,label_code,size,camera_lo,camera_la,light_color,drug_dir,back_color,drug_S
0,..\data\raw\sprint_ai_project1_data\train_anno...,K-001900,200,0,70,2,0,2,0
1,..\data\raw\sprint_ai_project1_data\train_anno...,K-001900,200,0,75,2,0,2,0
2,..\data\raw\sprint_ai_project1_data\train_anno...,K-001900,200,0,90,2,0,2,0
3,..\data\raw\sprint_ai_project1_data\train_anno...,K-016548,200,0,70,2,0,2,0
4,..\data\raw\sprint_ai_project1_data\train_anno...,K-016548,200,0,75,2,0,2,0



📊 데이터 분포 요약:
    drug_dir  light_color  camera_la
0      751.0          NaN        NaN
1       12.0          NaN        NaN
2        NaN        763.0        NaN
70       NaN          NaN      237.0
75       NaN          NaN      249.0
90       NaN          NaN      277.0


## 1.4 데이터 분포 확인 및 무결성 검증 (Verification)
- 생성된 데이터프레임(`df_meta`)을 기반으로 데이터가 한쪽으로 쏠려있지 않은지(Imbalance),

- 누락된 속성은 없는지 통계적으로 확인합니다.

- 이 수치는 팀원들에게 공유할 **초기 데이터 분석 보고서**의  근거 자료로 활용됩니다.

In [57]:
# [보고서 작성을 위한 통계 산출]
total_images = len(df_meta)
total_classes = df_meta['label_code'].nunique()
class_counts = df_meta['label_code'].value_counts()

# 1. 클래스 불균형 지표 (최대값 vs 최소값 비율)
max_count = class_counts.max()
min_count = class_counts.min()
imbalance_ratio = max_count / min_count

# 2. 메타데이터 분포 (촬영 각도, 조명 등 편향성 확인)
# 각 항목별로 데이터가 골고루 있는지 확인합니다.
angle_dist = df_meta['camera_la'].value_counts(normalize=True) * 100 # 퍼센트로 변환
light_dist = df_meta['light_color'].value_counts(normalize=True) * 100

print(f"--- [보고서용 핵심 수치] ---")
print(f"1. 총 데이터 개수: {total_images:,}장")
print(f"2. 총 약품(클래스) 종류: {total_classes}종")
print(f"3. 데이터 불균형 비율: 1 : {imbalance_ratio:.1f} (가장 적은 클래스 대비 가장 많은 클래스의 양)")
print(f"   - 최다 보유 약품: {class_counts.index[0]} ({max_count}장)")
print(f"   - 최소 보유 약품: {class_counts.index[-1]} ({min_count}장)")
print(f"4. 촬영 각도 분포:\n{angle_dist}")
print(f"--------------------------")

--- [보고서용 핵심 수치] ---
1. 총 데이터 개수: 763장
2. 총 약품(클래스) 종류: 56종
3. 데이터 불균형 비율: 1 : 51.0 (가장 적은 클래스 대비 가장 많은 클래스의 양)
   - 최다 보유 약품: K-003351 (153장)
   - 최소 보유 약품: K-027993 (3장)
4. 촬영 각도 분포:
camera_la
90    36.304063
75    32.634338
70    31.061599
Name: proportion, dtype: float64
--------------------------


In [61]:
# 진짜 결측치(Real Null)가 있는지 확인 (여기서 0이 나와야 진짜 깨끗한 데이터입니다!)
missing_check = df_meta.isnull().sum()

print("🔍 [결측치 점검 결과]")
if missing_check.sum() == 0:
    print("모든 데이터가 완벽합니다! (결측치 0건)")
else:
    print("결측치가 발견되었습니다:\n", missing_check[missing_check > 0])

print("-" * 30)

# 헷갈리지 않게 컬럼별로 따로 분포 출력하기
target_cols = ['drug_dir', 'light_color', 'camera_la', 'back_color', 'drug_S']

print("[데이터 분포 상세]")
for col in target_cols:
    print(f"\n📌 [{col}] 값 분포:")
    print(df_meta[col].value_counts().sort_index()) # 인덱스 순서대로 정렬해서 보기 좋게

🔍 [결측치 점검 결과]
모든 데이터가 완벽합니다! (결측치 0건)
------------------------------
[데이터 분포 상세]

📌 [drug_dir] 값 분포:
drug_dir
0    751
1     12
Name: count, dtype: int64

📌 [light_color] 값 분포:
light_color
2    763
Name: count, dtype: int64

📌 [camera_la] 값 분포:
camera_la
70    237
75    249
90    277
Name: count, dtype: int64

📌 [back_color] 값 분포:
back_color
2    763
Name: count, dtype: int64

📌 [drug_S] 값 분포:
drug_S
0    763
Name: count, dtype: int64


## 1.5 1차 EDA 결론 및 리포트
위 분석 과정을 통해 데이터의 물리적 구조와 메타데이터 스키마를 완전히 파악했습니다.

이를 정리하여 팀원들과 공유하기 위한 기술 문서(Technical Report)를 작성합니다.

> ### 📑 [Technical Report] 데이터셋 구성 및 분포 정밀 분석 결과

**작성자:** Data Engineer (안뜰에봄)  
**분석 일자:** 2026.01.31  
**분석 대상:** `sprint_ai_project1_data` (총 763장)

#### 1. 요약 (Executive Summary)
데이터 무결성 검사 및 파일명 파싱(Parsing)을 통해 메타데이터를 추출한 결과입니다.
* **총 데이터 수량:** **763장** (딥러닝 모델 학습에는 다소 **부족함**)

* **분류 대상(Class):** **56종** (약품 종류)

* **데이터 정합성:** 결측치(Null) **0건**으로 물리적 오류 없음.

> ##### **Insight:**
> 전체 데이터 양이 적고 클래스 불균형이 심해, 단순 학습 시 **과적합(Overfitting)** 및 **소수 클래스 인식 실패**가 우려됩니다.  
> 강력한 증강(Augmentation) 전략이 필수적입니다.

#### 2. 클래스 불균형(Class Imbalance) 심층 분석
- 가장 시급한 해결 과제인 데이터 불균형 현황입니다.

    | 구분 | 약품 코드(Label) | 데이터 수량 | 비중(%) |
    | :--- | :--- | :--- | :--- |
    | **최다(Max)** | `K-003351` | **153장** | 약 20.1% |
    | **최소(Min)** | `K-027993` | **3장** | 약 0.4% |
    | **불균형 비율** | - | **1 : 51.0** | - |


* **Risk** 
    1. 모델이 무조건 'K-003351'이라고 찍어도 20%의 정확도가 나오는 상황입니다.

    2. 반면 'K-027993'은 학습 기회가 3번밖에 없어 사실상 학습이 불가능합니다.

* **제안**
    1.  **데이터 증강:** 10장 미만인 소수 클래스(Minority Class)에 대해 `Rotate`, `Flip`, `Noise` 등을 적용하여 최소 50장 이상으로 **Oversampling** 필요.

    2.  **Loss Function:** 빈도수가 적은 클래스에 더 큰 페널티를 주는 `Focal Loss` 도입 권장.

#### 3. 촬영 환경(Metadata) 편향성 분석
- 모델이 실사용 환경에서 얼마나 잘 동작할지(Generalization) 가늠할 수 있는 지표입니다.

    * **촬영 각도(Camera Angle):**
        * 90도(Top): 36.3% / 75도: 32.6% / 70도: 31.1%

        * **판단:** 각도 분포는 매우 **이상적이고 균형적** 임.

    * **알약 방향(Direction):**
        * **앞면(0): 751장 (98.4%)** vs **뒷면(1): 12장 (1.6%)**

        * **Risk:** 뒷면으로 알약을 찍으면 모델이 인식 못 할 확률이 매우 높음.

    * **조명 및 배경(Fixed Condition):**
        * 조명(`light_color`)과 배경(`back_color`)이 모두 `2`번 코드로 **고정** 되어 있음.

        * **Risk:** 다른 조명이나 배경에서 촬영 시 성능 급락 예상. (Test 시 주의 필요)

#### 4. 향후 계획 (Action Plan)
1.  **전처리(Preprocessing):** 파일명 파싱 규칙을 적용한 `Custom Dataset` 클래스 구현.

2.  **데이터 스플릿(Split):** `StratifiedKFold`를 사용하여 학습/검증 셋을 나눌 때 소수 클래스(3장짜리 등)가 골고루 섞이도록 조치.

3.  **필수 구현 사항:** 학습 시 **뒷면 이미지(12장)** 에 가중치를 높게 설정하여 학습 빈도를 강제로 늘리는 로직 추가.